Please follow the rules/instructions for making the assignemnts. This means, for every exercise:

- Make sure the code is **well-documented**
- Update the README file, found in the same directory as the assignment, this means:
    - Describe the objective of the assignment
    - Provide instructions on how to run your code
    - Include any other relevant notes or dependencies

Also, please be mindful of the rules for the use of Generative AI:
- You may use generative AI tools (e.g., ChatGPT, GitHub Copilot, etc.) in your assignments under the following conditions:
    - Clearly indicate where and how you used generative AI
    - Save and submit the AI conversation alongside your assignment (upload the transcript alongside your assignment)
    - Remember: Using AI tools should support your learning, not replace it. Make sure the work you submit reflects your group’s understanding. All group members are still responsible for understanding and being able to explain all submitted content.